In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import os
from typing import Tuple

In [ ]:
def load_generators_train(dataset_dir: str, image_dim: Tuple[int, int], batch_size: int):
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        dataset_dir,
        shuffle=True,
        color_mode="rgb",
        class_mode="categorical",
        target_size=image_dim,
        batch_size=batch_size)
    # it takes input images and outputs augmented images

    print('created dataset for {}'.format(dataset_dir))

    return train_generator

In [ ]:
def load_generators_val(dataset_dir: str, image_dim: Tuple[int, int], batch_size: int):
    
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    train_generator = train_datagen.flow_from_directory(
        dataset_dir,
        shuffle=False,
        color_mode="rgb",
        class_mode="categorical",
        target_size=image_dim,
        batch_size=1)
    print('created dataset for {}'.format(dataset_dir))

    return train_generator

In [ ]:
BATCH_SIZE = 24
IMG_HEIGHT = 299
IMG_WIDTH = 299

train_dir = '/kaggle/input/coffee-dataset/coffee-datasets/symptom/train'
validation_dir = '/kaggle/input/coffee-dataset/coffee-datasets/symptom/val'

# load the data generators
train_datagen = load_generators_train(train_dir, (IMG_HEIGHT, IMG_WIDTH), BATCH_SIZE)

validation_datagen = load_generators_val(validation_dir, (IMG_HEIGHT, IMG_WIDTH), BATCH_SIZE)
#These new datasets are new transformed images produced using ImageDataGenerator class required for transfer learning.

In [ ]:
INCEPTIONV3_TFHUB = 'https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4'
#Getting the InceptionV3 model

In [ ]:
input_shape = (IMG_WIDTH, IMG_HEIGHT, 3)
num_classes = train_datagen.num_classes
#declaring the classes and input shape

# fetch the feature extractor from the tf_hub
feature_extractor = hub.KerasLayer(INCEPTIONV3_TFHUB, input_shape=input_shape)
#used to gather the transfer learning model

# make the feature extractor trainable
feature_extractor.trainable = True
# create the sequential model
model = tf.keras.Sequential([
    feature_extractor, 
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(num_classes, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.0005))
])

In [ ]:
# print the summary of the model
model.summary()

In [ ]:
# compiling the model
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.005,momentum=0.9),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# training the model
history=model.fit(
    train_datagen,
    epochs=12,
    steps_per_epoch=train_datagen.samples//train_datagen.batch_size,
    validation_data=validation_datagen)

In [ ]:
# evaluate the model
loss, accuracy = model.evaluate(validation_datagen)
# train accuracy
train_loss, train_accuracy = model.evaluate(train_datagen)

In [ ]:
"Trained Model for {} epochs, train accuracy: {:5.2f}%, test accuracy: {:5.2f}%".format(2, 100*train_accuracy, 100*accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
Y_pred = model.predict(validation_datagen)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
matrix=confusion_matrix(validation_datagen.classes, y_pred)
print(matrix)

In [ ]:
import matplotlib.pyplot as plt 
labels=['Healthy leaf','Leaf minor','Leaf rust','Phoma','Cercospora spot']
import seaborn as sns
figure = plt.figure(figsize=(8, 8))
sns.heatmap(matrix,annot=True,cmap=plt.cm.Blues,xticklabels=labels,yticklabels=labels)
plt.tight_layout()
plt.show()